In [1]:
import pymysql
import os

In [2]:
HOST = "rc1a-2f6hdfbbdf89l29u.mdb.yandexcloud.net"
PORT = 3306
USER = "alexey"
PASSWORD = "Password123"
DATABASE = "test_db"

ssl_ca = os.path.expanduser("~/.mysql/root.crt")

In [3]:
try:
    conn = pymysql.connect(
        host=HOST,
        port=PORT,
        user=USER,
        password=PASSWORD,
        database=DATABASE,
        ssl={'ca': ssl_ca},
        charset='utf8mb4',
        autocommit=True
    )
    cur = conn.cursor()
    cur.execute("SELECT VERSION(), DATABASE();")
    version, db = cur.fetchone()
    print(f"Версия MySQL: {version}")
    print(f"Текущая БД: {db}")
except Exception as e:
    print(f"{type(e).__name__}: {e}")

Версия MySQL: 8.0.41-32
Текущая БД: test_db


# 1. Создаём новую БД

In [7]:

new_db_name = "test_db_2"

# cur.execute(f"CREATE DATABASE IF NOT EXISTS `{new_db_name}`")
# print(f"{new_db_name} создана.") # в Yandex Cloud нельзя программно подкидывать новые бд

# Переключаемся на неё 
cur.execute(f"USE `{new_db_name}`;")
print(f"USE `{new_db_name}`")

cur.execute("SELECT VERSION(), DATABASE();")
version, db = cur.fetchone()
print(f"Версия MySQL: {version}")
print(f"Текущая БД: {db}")

USE `test_db_2`
Версия MySQL: 8.0.41-32
Текущая БД: test_db_2


2. Создание таблицы

In [8]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS characters (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        role VARCHAR(50),
        appearance_count INT
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
""")
print("таблица characters создана")

таблица characters создана


Вставляем данные

In [9]:
data = [
    ("Дориан Грей", "протагонист", 42),
    ("Лорд Генри Уоттон", "друг", 38),
    ("Бэзиль Холлуорд", "художник", 35),
    ("Портрет", "символ", 27),
    ("Леди Брэндон", "хозяйка салона", 3)
]

cur.executemany("""
    INSERT INTO characters (name, role, appearance_count)
    VALUES (%s, %s, %s);
""", data)

5

Работа с DataFrame через sqlalchemy

In [12]:
import pandas as pd

df = pd.read_csv('output/top10_words.csv')
print(df)

  Слово  Частота
0    не      107
1   что       79
2    вы       63
3    на       47
4    он       46
5   это       42
6    то       40
7   как       38
8   его       38
9    но       31


In [13]:
from sqlalchemy import create_engine

engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{new_db_name}",
    connect_args={"ssl": {"ca": ssl_ca}}
)

df.to_sql('top_words', con=engine, if_exists='replace', index=False)

10

In [17]:
chars_df = pd.read_sql("SELECT * FROM characters;", engine)
chars_df

,id,name,role,appearance_count
0,1,Дориан Грей,протагонист,42
1,2,Лорд Генри Уоттон,друг,38
2,3,Бэзиль Холлуорд,художник,35
3,4,Портрет,символ,27
4,5,Леди Брэндон,хозяйка салона,3


In [18]:
top2_df = pd.read_sql("SELECT * FROM top_words;", engine)
top2_df

,Слово,Частота
0,не,107
1,что,79
2,вы,63
3,на,47
4,он,46
5,это,42
6,то,40
7,как,38
8,его,38
9,но,31
